In [54]:
import heapq
from collections import defaultdict
import pprint

In [2]:
connections = [('A', 'B', 2), ('B', 'C', 4), ('B', 'D', 1), ('C', 'D', 6), ('E', 'F', 7), ('F', 'C', 5),
              ('A', 'E', 3),]

In [42]:
class Graph:
    
    def __init__(self, connections=None, directed=False):
        self._graph = defaultdict(set)
        self._directed = directed
        
        for connection in connections:
            self.add(connection[0], connection[1], connection[2])
    
    
    def add(self, node1, node2, weight):
        self._graph[node1].add((node2, weight))
        if not self._directed:
            self._graph[node2].add((node1, weight))
         
    def find_path(self, start, end, path=[]):
        ''' Find any path between start and end.  Not necessarily the shortest. '''
        
        path = path + [start]
        if start == end:
            return path
        
        for node in self._graph[start]:
            if node not in path:
                new_path = self.find_path(node, end, path)
                return new_path
            
    def dijkstras(self, start):
        # init cost to all nodes to infinity.
        cost_to_node = {node: float('inf') for node in self._graph}
        cost_to_node[start] = 0
        
        # To keep track of previously visited nodes.
        visited_nodes = set()
        
        pqueue = []
        for node, dist in cost_to_node.items():
            heapq.heappush(pqueue, (dist, node))
        
        # Loop through all nodes.
        while pqueue:
            # Pop nearest node.
            current_dist, current_node = heapq.heappop(pqueue)
            # Add node to visited list.
            visited_nodes.add(current_node)
            
            # Loop through neighbor nodes.
            for neighbor_node, neighbor_dist in self._graph[current_node]:
                # Skip nodes that have already been visited.
                if neighbor_node in visited_nodes:
                    continue
                    
                if current_dist + neighbor_dist < cost_to_node[neighbor_node]:
                    self._update_pqueue_priority(pqueue, 
                                                 (cost_to_node[neighbor_node], neighbor_node),
                                                 (current_dist + neighbor_dist, neighbor_node),
                                                )
                    cost_to_node[neighbor_node] = current_dist + neighbor_dist
                    
        return cost_to_node
                    
                    
    def _update_pqueue_priority(self, pqueue, current_node, replacement_node):
        index = pqueue.index(current_node)
        pqueue[index] = replacement_node
        heapq.heapify(pqueue)
        return pqueue
    
    def dfs(self, start):
        ''' Return set of nodes that can be reached using depth first search.
            This does not guarantee the shortest path.
        '''
        
        visited = {}
        visited[start] = start
        return self._dfs(start, visited)
        
        
    def _dfs(self, start, visited):
        ''' Utility function for DFS. '''
        
        
        for neighbor, weight in self._graph[start]:
            if neighbor not in visited:
                visited[neighbor] = start
                self._dfs(neighbor, visited)
        return visited
    
    def find_path_dfs(self, start, end):
        paths = self.dfs(start)
        
        if end not in paths:
            raise Exception('Cannot get to %s via %s' % (end, start))
        
        path = [end]
        via_node = paths[end]
        path.append(via_node)
        
        while via_node != start:
            via_node = paths[via_node]
            path.append(via_node)
            
        return path
        
        
    def bfs(self, start):
        
        
    
    def __repr__(self):
        return pprint.pformat(self._graph)
        

In [43]:
graph = Graph(connections, directed=False)

In [44]:
graph

defaultdict(<class 'set'>,
            {'A': {('E', 3), ('B', 2)},
             'B': {('D', 1), ('C', 4), ('A', 2)},
             'C': {('D', 6), ('F', 5), ('B', 4)},
             'D': {('C', 6), ('B', 1)},
             'E': {('F', 7), ('A', 3)},
             'F': {('C', 5), ('E', 7)}})

In [45]:
dfs_paths = graph.dfs('A')

In [46]:
for node, via in dfs_paths.items():
    print(node, via)

A A
E A
F E
C F
D C
B D


In [55]:
# A -> E -> F -> C -> D
graph.find_path_dfs('A', 'D')

['D', 'C', 'F', 'E', 'A']

In [56]:
q = graph.dijkstras('A')

In [57]:
q

{'A': 0, 'B': 2, 'C': 6, 'D': 3, 'E': 3, 'F': 10}